In [1]:
import pandas as pd
import requests
from census import Census
from us import states
from shapely.geometry import Point
import geopandas as gp
from scipy.stats import ttest_ind
import rtree
pd.set_option('display.max_columns', 500)

In [2]:
# To reproduce the data below, you'll need to save your 
# Census API key to `../data/census-api-key.txt`.
# You can obtain a key here: https://api.census.gov/data/key_signup.html
api_key = open("../data/census-api-key.txt").read().strip()
c = Census(api_key)

In [3]:
sf_bay_area = [
    {"state_code":"06", "county_code": "001", "county_name": "Alameda, CA"},
    {"state_code":"06", "county_code": "013", "county_name": "Contra Costa, CA"},
    {"state_code":"06", "county_code": "041", "county_name": "Marin, CA"},
    {"state_code":"06", "county_code": "055", "county_name": "Napa, CA"},
    {"state_code":"06", "county_code": "075", "county_name": "San Francisco, CA"},
    {"state_code":"06", "county_code": "081", "county_name": "San Mateo, CA"},
    {"state_code":"06", "county_code": "085", "county_name": "Santa Clara, CA"},
    {"state_code":"06", "county_code": "095", "county_name": "Solano, CA"},
    {"state_code":"06", "county_code": "097", "county_name": "Sonoma, CA"},
]

In [4]:
categories = [
     'NAME', # county name
     'B03002_001E', # Total population
     'B19013_001E', # Median income
     'B25077_001E', # Median home value
     'B15002_015E', # Total population age 25+ years with a bachelor's degree or higher MEN
     'B15002_032E', # Total population age 25+ years with a bachelor's degree or higher WOMEN
     'B15002_001E', # 25older
     'B03002_003E', # Not Hispanic or Latino!!White alone
     'B03002_004E', # Not Hispanic or Latino!!Black or African American alone
     'B03002_005E', # American Indian and Alaska Native Alone
     'B03002_006E', # Not Hispanic or Latino!!Asian alone
     'B03002_007E', # Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone
     'B03002_008E', # Not Hispanic or Latino!!Some other race alone
     'B03002_009E', # Not Hispanic or Latino!!Two or more races
     'B03002_012E', # Hispanic or Latino
     'B23025_005E', # Unemployed and in labor force
     'B23025_002E', # Labor force universe
     'B22001_002E', # receives SNAP
     'B22001_001E', # SNAP universe'
     'B19057_002E', # receives PAI
     'B19057_001E', # PAI universe(divide PAI with this)
     'B27001_001E', # health insurance universe
     'B27001_005E', 
     'B27001_008E', 
     'B27001_011E', 
     'B27001_014E', 
     'B27001_017E', 
     'B27001_020E', 
     'B27001_023E', 
     'B27001_026E',
     'B27001_029E', 
     'B27001_033E', 
     'B27001_036E', 
     'B27001_039E', 
     'B27001_042E', 
     'B27001_045E', 
     'B27001_048E', 
     'B27001_051E', 
     'B27001_054E', 
     'B27001_057E',
     'B25071_001E', # median percentage of income spent on rent
]

In [5]:
def get_acs_data(state_code, county_code):
    results = c.acs5.state_county_tract(
        categories,
        state_code,
        county_code, 
        Census.ALL,
        year = 2016
    )
    return [ {
        'geoid': res['state'] + res['county'] + res['tract'],
        'name': res['NAME'],
        'total_population': res['B03002_001E'],
        'median_income': res['B19013_001E'],
        'median_home_value': res['B25077_001E'],
        'educational_attainment': res['B15002_015E'] + res['B15002_032E'],
        '25older': res['B15002_001E'],
        'pct_edu': (res['B15002_015E'] + res['B15002_032E'])/res['B15002_001E']  if res['B15002_001E'] != 0 else 0,
        'white_alone': res['B03002_003E'],
        'pct_white': res['B03002_003E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'black_alone': res['B03002_004E'],
        'pct_black': res['B03002_004E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'native': res['B03002_005E'],
        'pct_native': res['B03002_005E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'asian': res['B03002_006E'],
        'pct_asian': res['B03002_006E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'native_hawaiian_pacific_islander': res['B03002_007E'],
        'pct_pac_isl': res['B03002_007E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'some_other_race_alone': res['B03002_008E'],
        'pct_other_race': res['B03002_008E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'two_or_more': res['B03002_009E'],
        'pct_two_race': res['B03002_009E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'hispanic_or_latino': res['B03002_012E'],
        'pct_hispanic': res['B03002_012E']/res['B03002_001E'] if res['B03002_001E'] != 0 else 0,
        'unemployed': res['B23025_005E'],
        'labor_force': res['B23025_002E'],
        'pct_unemp': res['B23025_005E']/res['B23025_002E'] if res['B23025_002E'] != 0 else 0,
        'receives_snap': res['B22001_002E'],
        'snap_universe': res['B22001_001E'],
        'pct_snap': res['B22001_002E']/res['B22001_001E'] if res['B22001_001E'] != 0 else 0,
        'receives_pai': res['B19057_002E'],
        'pai_universe': res['B19057_001E'],
        'pct_pai': res['B19057_002E']/res['B19057_001E'] if res['B19057_001E'] != 0 else 0,
        'uninsured': res['B27001_005E'] + res['B27001_008E'] + res['B27001_011E'] + res['B27001_014E'] + res['B27001_017E'] + res['B27001_020E'] + res['B27001_023E'] + res['B27001_026E'] + res['B27001_029E'] + res['B27001_033E'] + res['B27001_036E'] + res['B27001_039E'] + res['B27001_042E'] + res['B27001_045E'] + res['B27001_048E'] + res['B27001_051E'] + res['B27001_054E'] + res['B27001_057E'], 
        'insured_universe': res['B27001_001E'],
        'pct_uninsur': (res['B27001_005E'] + res['B27001_008E'] + res['B27001_011E'] + res['B27001_014E'] + res['B27001_017E'] + res['B27001_020E'] + res['B27001_023E'] + res['B27001_026E'] + res['B27001_029E'] + res['B27001_033E'] + res['B27001_036E'] + res['B27001_039E'] + res['B27001_042E'] + res['B27001_045E'] + res['B27001_048E'] + res['B27001_051E'] + res['B27001_054E'] + res['B27001_057E'])/res['B27001_001E'] if res['B27001_001E'] != 0 else 0,
        'median_percentage_income_rent': res['B25071_001E'],
    } for res in results ]

In [6]:
census_data = []
for county in sf_bay_area:
    print(county["county_name"])
    census_data += get_acs_data(county["state_code"], county["county_code"])

census_data = pd.DataFrame(census_data)[[
        'geoid',
        'name',
        'total_population',
        'median_income',
        'median_home_value',
        'educational_attainment',
        '25older',
        'pct_edu',
        'white_alone',
        'pct_white',
        'black_alone',
        'pct_black',
        'native',
        'pct_native',
        'asian',
        'pct_asian',
        'native_hawaiian_pacific_islander',
        'pct_pac_isl',
        'some_other_race_alone',
        'pct_other_race',
        'two_or_more',
        'pct_two_race',
        'hispanic_or_latino',
        'pct_hispanic',
        'unemployed',
        'labor_force',
        'pct_unemp',
        'receives_snap',
        'snap_universe',
        'pct_snap',
        'receives_pai',
        'pai_universe',
        'pct_pai',
        'uninsured', 
        'insured_universe',
        'pct_uninsur',
        'median_percentage_income_rent',
]]

census_data.head()

Alameda, CA
Contra Costa, CA
Marin, CA
Napa, CA
San Francisco, CA
San Mateo, CA
Santa Clara, CA
Solano, CA
Sonoma, CA


,geoid,name,total_population,median_income,median_home_value,educational_attainment,25older,pct_edu,white_alone,pct_white,black_alone,pct_black,native,pct_native,asian,pct_asian,native_hawaiian_pacific_islander,pct_pac_isl,some_other_race_alone,pct_other_race,two_or_more,pct_two_race,hispanic_or_latino,pct_hispanic,unemployed,labor_force,pct_unemp,receives_snap,snap_universe,pct_snap,receives_pai,pai_universe,pct_pai,uninsured,insured_universe,pct_uninsur,median_percentage_income_rent
0,06001400100,"Census Tract 4001, Alameda County, California",3018.0,177417.0,1074100.0,781.0,2478.0,0.315174,2145.0,0.710736,92.0,0.030484,0.0,0.000000,491.0,0.162691,0.0,0.000000,75.0,0.024851,132.0,0.043738,83.0,0.027502,75.0,1643.0,0.045648,0.0,1292.0,0.000000,9.0,1292.0,0.006966,110.0,3018.0,0.036448,17.6
1,06001400200,"Census Tract 4002, Alameda County, California",1960.0,153125.0,978900.0,557.0,1559.0,0.357280,1426.0,0.727551,11.0,0.005612,4.0,0.002041,165.0,0.084184,0.0,0.000000,5.0,0.002551,138.0,0.070408,211.0,0.107653,41.0,1270.0,0.032283,5.0,813.0,0.006150,0.0,813.0,0.000000,73.0,1960.0,0.037245,18.5
2,06001400300,"Census Tract 4003, Alameda County, California",5236.0,85313.0,912700.0,1332.0,4124.0,0.322987,3346.0,0.639037,606.0,0.115737,9.0,0.001719,599.0,0.114400,36.0,0.006875,0.0,0.000000,254.0,0.048510,386.0,0.073720,204.0,3402.0,0.059965,55.0,2439.0,0.022550,58.0,2439.0,0.023780,160.0,5236.0,0.030558,27.0
3,06001400400,"Census Tract 4004, Alameda County, California",4171.0,99539.0,848900.0,1123.0,3303.0,0.339994,2758.0,0.661232,410.0,0.098298,13.0,0.003117,458.0,0.109806,0.0,0.000000,0.0,0.000000,266.0,0.063774,266.0,0.063774,114.0,2678.0,0.042569,29.0,1798.0,0.016129,27.0,1798.0,0.015017,173.0,4165.0,0.041537,23.7
4,06001400500,"Census Tract 4005, Alameda County, California",3748.0,83650.0,683500.0,1053.0,2943.0,0.357798,1972.0,0.526147,944.0,0.251868,0.0,0.000000,140.0,0.037353,24.0,0.006403,19.0,0.005069,163.0,0.043490,486.0,0.129669,82.0,2545.0,0.032220,50.0,1643.0,0.030432,71.0,1643.0,0.043214,397.0,3748.0,0.105923,24.3


In [7]:
len(census_data)

1588

In [8]:
census_data.head()

,geoid,name,total_population,median_income,median_home_value,educational_attainment,25older,pct_edu,white_alone,pct_white,black_alone,pct_black,native,pct_native,asian,pct_asian,native_hawaiian_pacific_islander,pct_pac_isl,some_other_race_alone,pct_other_race,two_or_more,pct_two_race,hispanic_or_latino,pct_hispanic,unemployed,labor_force,pct_unemp,receives_snap,snap_universe,pct_snap,receives_pai,pai_universe,pct_pai,uninsured,insured_universe,pct_uninsur,median_percentage_income_rent
0,06001400100,"Census Tract 4001, Alameda County, California",3018.0,177417.0,1074100.0,781.0,2478.0,0.315174,2145.0,0.710736,92.0,0.030484,0.0,0.000000,491.0,0.162691,0.0,0.000000,75.0,0.024851,132.0,0.043738,83.0,0.027502,75.0,1643.0,0.045648,0.0,1292.0,0.000000,9.0,1292.0,0.006966,110.0,3018.0,0.036448,17.6
1,06001400200,"Census Tract 4002, Alameda County, California",1960.0,153125.0,978900.0,557.0,1559.0,0.357280,1426.0,0.727551,11.0,0.005612,4.0,0.002041,165.0,0.084184,0.0,0.000000,5.0,0.002551,138.0,0.070408,211.0,0.107653,41.0,1270.0,0.032283,5.0,813.0,0.006150,0.0,813.0,0.000000,73.0,1960.0,0.037245,18.5
2,06001400300,"Census Tract 4003, Alameda County, California",5236.0,85313.0,912700.0,1332.0,4124.0,0.322987,3346.0,0.639037,606.0,0.115737,9.0,0.001719,599.0,0.114400,36.0,0.006875,0.0,0.000000,254.0,0.048510,386.0,0.073720,204.0,3402.0,0.059965,55.0,2439.0,0.022550,58.0,2439.0,0.023780,160.0,5236.0,0.030558,27.0
3,06001400400,"Census Tract 4004, Alameda County, California",4171.0,99539.0,848900.0,1123.0,3303.0,0.339994,2758.0,0.661232,410.0,0.098298,13.0,0.003117,458.0,0.109806,0.0,0.000000,0.0,0.000000,266.0,0.063774,266.0,0.063774,114.0,2678.0,0.042569,29.0,1798.0,0.016129,27.0,1798.0,0.015017,173.0,4165.0,0.041537,23.7
4,06001400500,"Census Tract 4005, Alameda County, California",3748.0,83650.0,683500.0,1053.0,2943.0,0.357798,1972.0,0.526147,944.0,0.251868,0.0,0.000000,140.0,0.037353,24.0,0.006403,19.0,0.005069,163.0,0.043490,486.0,0.129669,82.0,2545.0,0.032220,50.0,1643.0,0.030432,71.0,1643.0,0.043214,397.0,3748.0,0.105923,24.3


In [9]:
census_data['geoid'] = census_data['geoid'].astype(str)

In [10]:
(
    census_data
    .assign(
        state_code = lambda df: df["geoid"].str.slice(0, 2),
        county_code = lambda df: df["geoid"].str.slice(2, 5)
    )
    .groupby([
        "state_code",
        "county_code"
    ])
    .size()
    .to_frame("tracts")
    .reset_index()
    .merge(
        pd.DataFrame(sf_bay_area),
        how = "outer",
        on = [
            "state_code",
            "county_code"
        ]
    )
    .sort_values("tracts", ascending = False)
)

,state_code,county_code,tracts,county_name
6,06,085,372,"Santa Clara, CA"
0,06,001,361,"Alameda, CA"
1,06,013,208,"Contra Costa, CA"
4,06,075,197,"San Francisco, CA"
5,06,081,158,"San Mateo, CA"
8,06,097,100,"Sonoma, CA"
7,06,095,96,"Solano, CA"
2,06,041,56,"Marin, CA"
3,06,055,40,"Napa, CA"


In [12]:
ca_tracts = (
    gp.read_file(
        "../data/cb_2016_06_tract_500k/cb_2016_06_tract_500k.shp"
    )
    .drop(columns = [
        "TRACTCE",
        "AFFGEOID",
        "NAME",
        "ALAND",
        "AWATER",
        "LSAD"
    ])
)

In [13]:
county_codes = [i['county_code'] for i in sf_bay_area]
county_codes

['001', '013', '041', '055', '075', '081', '085', '095', '097']

In [14]:
bay_area_tracts = ca_tracts[
    ca_tracts['COUNTYFP'].isin(county_codes)
]
tract_geoids = set(bay_area_tracts['GEOID'].values)
len(tract_geoids)

1582

In [15]:
census_data = census_data.rename(index=str, columns={"geoid": "GEOID"})

In [16]:
merged = bay_area_tracts.merge(census_data, on='GEOID')

In [17]:
merged.to_file("../data/merged_bay_area_tracts.shp")

In [75]:
census_data

,GEOID,name,total_population,median_income,median_home_value,educational_attainment,25older,pct_edu,white_alone,pct_white,black_alone,pct_black,native,pct_native,asian,pct_asian,native_hawaiian_pacific_islander,pct_pac_isl,some_other_race_alone,pct_other_race,two_or_more,pct_two_race,hispanic_or_latino,pct_hispanic,unemployed,labor_force,pct_unemp,receives_snap,snap_universe,pct_snap,receives_pai,pai_universe,pct_pai,uninsured,insured_universe,pct_uninsur,median_percentage_income_rent
0,06001400100,"Census Tract 4001, Alameda County, California",3018.0,177417.0,1074100.0,781.0,2478.0,0.315174,2145.0,0.710736,92.0,0.030484,0.0,0.000000,491.0,0.162691,0.0,0.000000,75.0,0.024851,132.0,0.043738,83.0,0.027502,75.0,1643.0,0.045648,0.0,1292.0,0.000000,9.0,1292.0,0.006966,110.0,3018.0,0.036448,17.6
1,06001400200,"Census Tract 4002, Alameda County, California",1960.0,153125.0,978900.0,557.0,1559.0,0.357280,1426.0,0.727551,11.0,0.005612,4.0,0.002041,165.0,0.084184,0.0,0.000000,5.0,0.002551,138.0,0.070408,211.0,0.107653,41.0,1270.0,0.032283,5.0,813.0,0.006150,0.0,813.0,0.000000,73.0,1960.0,0.037245,18.5
2,06001400300,"Census Tract 4003, Alameda County, California",5236.0,85313.0,912700.0,1332.0,4124.0,0.322987,3346.0,0.639037,606.0,0.115737,9.0,0.001719,599.0,0.114400,36.0,0.006875,0.0,0.000000,254.0,0.048510,386.0,0.073720,204.0,3402.0,0.059965,55.0,2439.0,0.022550,58.0,2439.0,0.023780,160.0,5236.0,0.030558,27.0
3,06001400400,"Census Tract 4004, Alameda County, California",4171.0,99539.0,848900.0,1123.0,3303.0,0.339994,2758.0,0.661232,410.0,0.098298,13.0,0.003117,458.0,0.109806,0.0,0.000000,0.0,0.000000,266.0,0.063774,266.0,0.063774,114.0,2678.0,0.042569,29.0,1798.0,0.016129,27.0,1798.0,0.015017,173.0,4165.0,0.041537,23.7
4,06001400500,"Census Tract 4005, Alameda County, California",3748.0,83650.0,683500.0,1053.0,2943.0,0.357798,1972.0,0.526147,944.0,0.251868,0.0,0.000000,140.0,0.037353,24.0,0.006403,19.0,0.005069,163.0,0.043490,486.0,0.129669,82.0,2545.0,0.032220,50.0,1643.0,0.030432,71.0,1643.0,0.043214,397.0,3748.0,0.105923,24.3
5,06001400600,"Census Tract 4006, Alameda County, California",1661.0,61597.0,572300.0,407.0,1233.0,0.330089,636.0,0.382902,584.0,0.351595,5.0,0.003010,116.0,0.069837,0.0,0.000000,19.0,0.011439,143.0,0.086093,158.0,0.095123,104.0,1116.0,0.093190,43.0,713.0,0.060309,38.0,713.0,0.053296,151.0,1661.0,0.090909,36.8
6,06001400700,"Census Tract 4007, Alameda County, California",4552.0,46424.0,608500.0,853.0,3467.0,0.246034,1700.0,0.373462,1691.0,0.371485,75.0,0.016476,119.0,0.026142,0.0,0.000000,0.0,0.000000,215.0,0.047232,752.0,0.165202,227.0,2712.0,0.083702,215.0,1787.0,0.120313,33.0,1787.0,0.018467,692.0,4552.0,0.152021,32.5
7,06001400800,"Census Tract 4008, Alameda County, California",3506.0,61111.0,535600.0,832.0,2821.0,0.294931,1390.0,0.396463,1093.0,0.311751,7.0,0.001997,436.0,0.124358,0.0,0.000000,0.0,0.000000,159.0,0.045351,421.0,0.120080,203.0,2296.0,0.088415,84.0,1580.0,0.053165,59.0,1580.0,0.037342,362.0,3500.0,0.103429,30.0
8,06001400900,"Census Tract 4009, Alameda County, California",2262.0,55691.0,505100.0,498.0,1712.0,0.290888,899.0,0.397436,734.0,0.324492,10.0,0.004421,188.0,0.083112,0.0,0.000000,63.0,0.027851,104.0,0.045977,264.0,0.116711,62.0,1435.0,0.043206,60.0,1025.0,0.058537,39.0,1025.0,0.038049,146.0,2262.0,0.064545,30.0
9,06001401000,"Census Tract 4010, Alameda County, California",6193.0,51073.0,422100.0,877.0,4464.0,0.196461,1914.0,0.309059,2057.0,0.332149,9.0,0.001453,194.0,0.031326,85.0,0.013725,56.0,0.009042,498.0,0.080413,1380.0,0.222832,333.0,3954.0,0.084219,299.0,2619.0,0.114166,248.0,2619.0,0.094693,574.0,6184.0,0.092820,36.4


In [79]:
merged['pct_snap'].max()

0.415907207953604

In [80]:
merged['pct_snap'].describe()

count    1582.000000
mean        0.063209
std         0.069480
min         0.000000
25%         0.014011
50%         0.038639
75%         0.087295
max         0.415907
Name: pct_snap, dtype: float64

In [81]:
merged['pct_snap'].corr(merged['pct_black'])

0.5026067044882905

In [82]:
merged['pct_snap'].corr(merged['pct_white'])

-0.5598262081514863

In [87]:
merged['pct_snap'].corr(merged['pct_pai'])

0.7430176174715474

In [88]:
merged['pct_snap'].corr(merged['pct_uninsur'])

0.6525976529774126

In [89]:
merged['pct_snap'].corr(merged['pct_unemp'])

0.6208111925076788

In [91]:
merged['pct_snap'].corr(merged['pct_hispanic'])

0.6458775658699161

In [90]:
merged['pct_snap'].corr(merged['median_percentage_income_rent'])

0.05476526855446865